In [9]:
import sympy as sp
import numpy as np

In [10]:
class EKF:
    
    def __init__(self, x0, P0, Q, R, updateX, updateY, JacX, JacY):
        self.X_prev = x0
        self.P = P0
        self.Q = Q
        self.R = R
        self.updateX = updateX
        self.updateY = updateY
        self.JacX = JacX
        self.JacY = JacY
    

In [11]:
q0, q1, q2, q3 = sp.symbols(r"q_{0} q_{1} q_{2} q_{3}")
pitch, roll, yaw = sp.symbols(r" \theta \phi \psi")
p, q, r = sp.symbols("p q r")
dt = sp.symbols(r"\delta_{T}")

Bx, By, Bz = sp.symbols(r"B_{x} B_{y} B_{z}")

X = sp.Matrix([q0, q1, q2, q3])
U = sp.Matrix([p, q, r])

g = sp.Matrix([0, 0, 1])
B = sp.Matrix([Bx, By, Bz])

In [14]:
def skewSym(U):
    p, q, r = U[0], U[1], U[2]
    w = sp.Matrix([[0, -p, -q, -r],
                   [p, 0, r, -q],
                   [q, -r, 0, p],
                   [r, q, -p, 0]])
    return w

def updateX(X, U):
    X_next = 0.5 * skewSym(U) @ X
    X_next = sp.simplify(X_next)
    return X_next
    
def JacX(X, U):
    X_next = X + updateX(X, U) * dt
    jacX = X_next.jacobian(X) 
    jacX = sp.simplify(jacX)
    return jacX

def quatDCM(X):
    q0, q1, q2, q3 = X
    DCM = sp.zeros(3,3)
    DCM[0,0] = q0*q0 + q1*q1 - q2*q2 - q3*q3
    DCM[0,1] = 2 * (q1*q2 + q0*q3)
    DCM[0,2] = 2 * (q1*q3 - q0*q2)
    DCM[1,0] = 2 * (q1*q2 - q0*q3)
    DCM[1,1] = q0*q0 - q1*q1 + q2*q2 - q3*q3
    DCM[1,2] = 2 * (q2*q3 + q0*q1)
    DCM[2,0] = 2 * (q1*q3 + q0*q2)
    DCM[2,1] = 2 * (q2*q3 - q0*q1)
    DCM[2,2] = q0*q0 - q1*q1 - q2*q2 + q3*q3
    return DCM

def updateY(X, U):
    Y = sp.zeros(6,1)
    DCM = quatDCM(X)
    Y[0:3, 0] = DCM @ g
    Y[3:, 0] = DCM @ B
    return Y

def JacY(X, U):
    Y = updateY(X, U)
    jacY = Y.jacobian(X)
    return sp.simplify(jacY)

In [ ]:
updateX = sp.lambdify([X,U])